<a href="https://colab.research.google.com/github/Anvar-baba/pythonProject-7/blob/main/%D0%9F%D0%BE%D0%B3%D0%BE%D0%B4%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Установка библиотек
!pip install requests pandas matplotlib seaborn plotly -q

import requests
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

# Настройки
plt.rcParams['figure.figsize'] = (12, 6)
sns.set_style("whitegrid")

class WeatherService:
    def __init__(self):
        self.base_url = "http://wttr.in/"

    def get_current_weather(self, city_name):
        """Получение текущей погоды"""
        try:
            # Получаем данные в JSON формате
            url = f"{self.base_url}/{city_name}?format=j1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()

                current_condition = data['current_condition'][0]
                weather_data = {
                    'Город': city_name,
                    'Температура': f"{current_condition['temp_C']}°C",
                    'Ощущается как': f"{current_condition['FeelsLikeC']}°C",
                    'Влажность': f"{current_condition['humidity']}%",
                    'Давление': f"{current_condition['pressure']} hPa",
                    'Ветер': f"{current_condition['windspeedKmph']} км/ч",
                    'Направление': current_condition['winddir16Point'],
                    'Описание': current_condition['weatherDesc'][0]['value'],
                    'Видимость': f"{current_condition['visibility']} км",
                    'UV индекс': current_condition['uvIndex']
                }
                return pd.DataFrame([weather_data])
            else:
                return f"❌ Ошибка: Не удалось получить данные (код {response.status_code})"

        except Exception as e:
            return f"❌ Ошибка: {str(e)}"

    def get_weather_forecast(self, city_name, days=3):
        """Получение прогноза погоды"""
        try:
            url = f"{self.base_url}/{city_name}?format=j1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                forecast_data = []

                # Берем указанное количество дней (максимум 3, т.к. wttr.in дает 3 дня)
                days = min(days, 3)

                for i in range(days):
                    day_data = data['weather'][i]
                    date = day_data['date']

                    # Средняя температура за день
                    avg_temp = (int(day_data['mintempC']) + int(day_data['maxtempC'])) // 2

                    forecast_data.append({
                        'Дата': date,
                        'День недели': ['Пн', 'Вт', 'Ср', 'Чт', 'Пт', 'Сб', 'Вс'][datetime.strptime(date, '%Y-%m-%d').weekday()],
                        'Мин. температура': f"{day_data['mintempC']}°C",
                        'Макс. температура': f"{day_data['maxtempC']}°C",
                        'Средняя температура': f"{avg_temp}°C",
                        'Восход': day_data['astronomy'][0]['sunrise'],
                        'Закат': day_data['astronomy'][0]['sunset'],
                        'Влажность': f"{day_data['hourly'][4]['humidity']}%",
                        'Ветер': f"{day_data['hourly'][4]['windspeedKmph']} км/ч",
                        'Описание': day_data['hourly'][4]['weatherDesc'][0]['value']
                    })

                return pd.DataFrame(forecast_data)
            else:
                return f"❌ Ошибка: Не удалось получить прогноз (код {response.status_code})"

        except Exception as e:
            return f"❌ Ошибка при получении прогноза: {str(e)}"

    def get_hourly_forecast(self, city_name, hours=24):
        """Получение почасового прогноза"""
        try:
            url = f"{self.base_url}/{city_name}?format=j1"
            response = requests.get(url, timeout=10)

            if response.status_code == 200:
                data = response.json()
                hourly_data = []

                # Берем почасовые данные на сегодня
                today_hours = data['weather'][0]['hourly']
                hours = min(hours, 8)  # wttr.in дает 8 периодов на день

                for i in range(hours):
                    hour_data = today_hours[i]
                    time = f"{int(hour_data['time']):02d}:00"

                    hourly_data.append({
                        'Время': time,
                        'Температура': f"{hour_data['tempC']}°C",
                        'Ощущается как': f"{hour_data['FeelsLikeC']}°C",
                        'Влажность': f"{hour_data['humidity']}%",
                        'Давление': f"{hour_data['pressure']} hPa",
                        'Ветер': f"{hour_data['windspeedKmph']} км/ч",
                        'Вероятность дождя': f"{hour_data['chanceofrain']}%",
                        'Описание': hour_data['weatherDesc'][0]['value']
                    })

                return pd.DataFrame(hourly_data)
            else:
                return f"❌ Ошибка почасового прогноза"

        except Exception as e:
            return f"❌ Ошибка: {str(e)}"

def create_temperature_plot(daily_forecast, city_name):
    """Создание графика температуры"""
    if not isinstance(daily_forecast, pd.DataFrame):
        return

    try:
        # Извлекаем числовые значения температур
        min_temps = daily_forecast['Мин. температура'].str.replace('°C', '').astype(int)
        max_temps = daily_forecast['Макс. температура'].str.replace('°C', '').astype(int)

        fig = go.Figure()

        # Максимальные температуры
        fig.add_trace(go.Scatter(
            x=daily_forecast['Дата'],
            y=max_temps,
            mode='lines+markers+text',
            name='Максимум',
            line=dict(color='red', width=3),
            marker=dict(size=8),
            text=max_temps,
            textposition='top center'
        ))

        # Минимальные температуры
        fig.add_trace(go.Scatter(
            x=daily_forecast['Дата'],
            y=min_temps,
            mode='lines+markers+text',
            name='Минимум',
            line=dict(color='blue', width=3),
            marker=dict(size=8),
            text=min_temps,
            textposition='bottom center'
        ))

        fig.update_layout(
            title=f'📈 Прогноз температуры в {city_name}',
            xaxis_title='Дата',
            yaxis_title='Температура (°C)',
            template='plotly_white',
            height=500
        )

        fig.show()
    except Exception as e:
        print(f"Ошибка при построении графика: {e}")

def create_hourly_plot(hourly_forecast, city_name):
    """Создание почасового графика"""
    if not isinstance(hourly_forecast, pd.DataFrame):
        return

    try:
        temps = hourly_forecast['Температура'].str.replace('°C', '').astype(int)

        fig = go.Figure()

        fig.add_trace(go.Scatter(
            x=hourly_forecast['Время'],
            y=temps,
            mode='lines+markers',
            name='Температура',
            line=dict(color='orange', width=3),
            marker=dict(size=6),
            fill='tozeroy',
            fillcolor='rgba(255,165,0,0.1)'
        ))

        fig.update_layout(
            title=f'🌡️ Почасовой прогноз в {city_name}',
            xaxis_title='Время',
            yaxis_title='Температура (°C)',
            template='plotly_white',
            height=400
        )

        fig.show()
    except Exception as e:
        print(f"Ошибка при построении почасового графика: {e}")

def create_weather_dashboard():
    """Создание интерактивной панели"""

    # Виджеты
    city_input = widgets.Text(
        value='Москва',
        placeholder='Введите город (Moscow, London, Paris...)',
        description='🏙️ Город:',
        disabled=False,
        layout=widgets.Layout(width='300px')
    )

    forecast_type = widgets.Dropdown(
        options=['Текущая погода', 'Прогноз на 3 дня', 'Почасовой прогноз'],
        value='Текущая погода',
        description='📊 Тип:',
        disabled=False
    )

    button = widgets.Button(
        description="🌤️ Получить погоду",
        button_style='success',
        icon='check',
        layout=widgets.Layout(width='200px', height='40px')
    )

    output = widgets.Output()

    def on_button_click(b):
        with output:
            clear_output()
            weather_service = WeatherService()

            print("🔍 Получаем данные о погоде...")

            if forecast_type.value == 'Текущая погода':
                current_weather = weather_service.get_current_weather(city_input.value)

                if isinstance(current_weather, pd.DataFrame):
                    print("\n📊 ТЕКУЩАЯ ПОГОДА:")
                    # Транспонируем для лучшего отображения
                    styled_df = current_weather.T
                    styled_df.columns = ['Значение']
                    display(styled_df.style.set_properties(**{
                        'background-color': '#e8f4fd',
                        'border': '1px solid #b3d9ff',
                        'font-size': '14px',
                        'padding': '8px'
                    }))
                else:
                    print(f"\n{current_weather}")

            elif forecast_type.value == 'Прогноз на 3 дня':
                forecast = weather_service.get_weather_forecast(city_input.value, 3)

                if isinstance(forecast, pd.DataFrame):
                    print("\n📈 ПРОГНОЗ НА 3 ДНЯ:")
                    display(forecast.style.set_properties(**{
                        'background-color': '#f0f8ff',
                        'border': '1px solid #d3d3d3'
                    }))

                    create_temperature_plot(forecast, city_input.value)
                else:
                    print(f"\n{forecast}")

            elif forecast_type.value == 'Почасовой прогноз':
                hourly = weather_service.get_hourly_forecast(city_input.value, 8)

                if isinstance(hourly, pd.DataFrame):
                    print("\n🕒 ПОЧАСОВОЙ ПРОГНОЗ (СЕГОДНЯ):")
                    display(hourly.style.set_properties(**{
                        'background-color': '#fff0f5',
                        'border': '1px solid #d3d3d3'
                    }))

                    create_hourly_plot(hourly, city_input.value)
                else:
                    print(f"\n{hourly}")

    button.on_click(on_button_click)

    # Отображение
    display(widgets.VBox([
        widgets.HTML("<h1 style='text-align: center; color: #1e88e5;'>🌤️ ПАРСЕР ПОГОДЫ (wttr.in)</h1>"),
        widgets.HTML("<p style='text-align: center;'>Работает в Google Colab • Данные с wttr.in</p>"),
        widgets.HBox([city_input, forecast_type]),
        widgets.HBox([button]),
        output
    ]))

# Простая версия для быстрого теста
def quick_weather_test():
    """Быстрая проверка работы сервиса"""
    weather_service = WeatherService()

    print("🌍 Тестируем сервис погоды...")
    print("Доступные форматы городов:")
    print("• Москва, London, Paris, Berlin")
    print("• New York, Tokyo, Dubai, Istanbul")
    print("• И многие другие...\n")

    city = input("Введите город для проверки: ").strip()
    if not city:
        city = "Москва"

    print(f"\n🔄 Тестируем для города: {city}")

    # Тест текущей погоды
    print("\n1. Текущая погода:")
    current = weather_service.get_current_weather(city)
    if isinstance(current, pd.DataFrame):
        print("✅ Сервис работает!")
        for col in current.columns:
            print(f"   {col}: {current[col].iloc[0]}")
    else:
        print(f"❌ {current}")

    # Тест прогноза
    print("\n2. Прогноз на 3 дня:")
    forecast = weather_service.get_weather_forecast(city, 3)
    if isinstance(forecast, pd.DataFrame):
        print("✅ Прогноз получен!")
        print(forecast[['Дата', 'Мин. температура', 'Макс. температура', 'Описание']].to_string(index=False))
    else:
        print(f"❌ {forecast}")

# Информация о сервисе
def show_service_info():
    print("\n💡 О сервисе:")
    print("   • Используется wttr.in - бесплатный сервис погоды")
    print("   • Не требует API ключей")
    print("   • Работает по всему миру")
    print("   • Обновляется в реальном времени")
    print("   • Поддерживает города на разных языках\n")

# Главное меню
print("=" * 60)
print("🌤️  ПАРСЕР ПОГОДЫ ДЛЯ GOOGLE COLAB")
print("=" * 60)

show_service_info()

print("Выберите режим работы:")
print("1️⃣  - Интерактивная панель (рекомендуется)")
print("2️⃣  - Быстрый тест сервиса")
print("3️⃣  - Проверка подключения")

choice = input("Ваш выбор (1/2/3): ").strip()

if choice == "1":
    create_weather_dashboard()
elif choice == "2":
    quick_weather_test()
elif choice == "3":
    print("\n🔍 Проверяем подключение к wttr.in...")
    try:
        response = requests.get("http://wttr.in/Moscow?format=j1", timeout=10)
        if response.status_code == 200:
            print("✅ Подключение к wttr.in работает отлично!")
            print("✅ Сервис погоды доступен")
        else:
            print(f"❌ Ошибка: код {response.status_code}")
    except Exception as e:
        print(f"❌ Ошибка подключения: {e}")
else:
    print("Запускаю интерактивную панель...")
    create_weather_dashboard()

🌤️  ПАРСЕР ПОГОДЫ ДЛЯ GOOGLE COLAB

💡 О сервисе:
   • Используется wttr.in - бесплатный сервис погоды
   • Не требует API ключей
   • Работает по всему миру
   • Обновляется в реальном времени
   • Поддерживает города на разных языках

Выберите режим работы:
1️⃣  - Интерактивная панель (рекомендуется)
2️⃣  - Быстрый тест сервиса
3️⃣  - Проверка подключения
Ваш выбор (1/2/3): 1
